In [ ]:
# Montar Google Drive y configurar el entorno
from google.colab import drive
drive.mount('/content/drive')

# Instalación de bibliotecas necesarias
!pip install transformers datasets torch evaluate scikit-learn pandas numpy seaborn matplotlib mlflow wandb nltk tqdm

# Importaciones básicas
import pandas as pd
import numpy as np
import torch
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix
)
# Añadir esta importación
from sklearn.utils.class_weight import compute_class_weight  # <- Añadir esta línea
import evaluate
import seaborn as sns
import matplotlib.pyplot as plt
import logging
import json
from datetime import datetime
import mlflow
import os
import dataclasses
import torch.cuda.amp
import re
import nltk
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')


# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('training.log'),
        logging.StreamHandler()
    ]
)

# Configuración de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Cargar los datos
def load_and_preprocess_data():
    logging.info("Iniciando carga y preprocesamiento de datos...")

    # Cargar dataset
    df = pd.read_csv('/content/drive/MyDrive/IronHack/Proyecto4/nlp-project/data/processed/consolidated_reviews.csv')
    logging.info(f"Dataset cargado con {len(df)} registros")

    # Limpieza básica
    df_cleaned = df.copy()
    df_cleaned = df_cleaned.dropna(subset=['reviews.text'])

    # Crear mapeo de etiquetas
    label_mapping = {'positive': 0, 'neutral': 1, 'negative': 2}
    df_cleaned['label'] = df_cleaned['sentiment'].map(label_mapping)

    # Calcular pesos de clase para manejar el desbalance
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(df_cleaned['label']),
        y=df_cleaned['label']
    )
    class_weights = torch.FloatTensor(class_weights).to(device)

    # División de datos
    train_df, test_df = train_test_split(
        df_cleaned,
        test_size=0.2,
        random_state=42,
        stratify=df_cleaned['sentiment']
    )
    train_df, val_df = train_test_split(
        train_df,
        test_size=0.2,
        random_state=42,
        stratify=train_df['sentiment']
    )

    # Mostrar distribución de clases
    print_class_distribution(train_df, val_df, test_df)

    return train_df, val_df, test_df, class_weights, label_mapping

def print_class_distribution(train_df, val_df, test_df):
    print("Distribución de clases en cada conjunto:")
    print("\nEntrenamiento:")
    print(train_df['sentiment'].value_counts(normalize=True))
    print("\nValidación:")
    print(val_df['sentiment'].value_counts(normalize=True))
    print("\nPrueba:")
    print(test_df['sentiment'].value_counts(normalize=True))

# Clase para el dataset de PyTorch
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

def create_datasets(train_df, val_df, test_df, tokenizer):
    logging.info("Creando datasets de PyTorch...")

    # Crear los datasets
    train_dataset = ReviewDataset(
        train_df['reviews.text'].tolist(),
        train_df['label'].tolist(),
        tokenizer
    )
    val_dataset = ReviewDataset(
        val_df['reviews.text'].tolist(),
        val_df['label'].tolist(),
        tokenizer
    )
    test_dataset = ReviewDataset(
        test_df['reviews.text'].tolist(),
        test_df['label'].tolist(),
        tokenizer
    )

    # Verificar las dimensiones
    logging.info(f"Tamaño del conjunto de entrenamiento: {len(train_dataset)}")
    logging.info(f"Tamaño del conjunto de validación: {len(val_dataset)}")
    logging.info(f"Tamaño del conjunto de prueba: {len(test_dataset)}")

    return train_dataset, val_dataset, test_dataset

def analyze_text_lengths(train_df, tokenizer):
    reviews_length = train_df['reviews.text'].str.split().str.len()
    print("Estadísticas de longitud de las reseñas:")
    print(f"Media: {reviews_length.mean():.2f}")
    print(f"Mediana: {reviews_length.median():.2f}")
    print(f"95 percentil: {reviews_length.quantile(0.95):.2f}")
    print(f"Máximo: {reviews_length.max():.2f}")

    # Ejemplo de tokenización
    sample_text = train_df['reviews.text'].iloc[0]
    sample_tokens = tokenizer.encode(sample_text, truncation=True)
    print("\nEjemplo de tokenización:")
    print(f"Texto original: {sample_text}")
    print(f"Número de tokens: {len(sample_tokens)}")
    print(f"Tokens decodificados: {tokenizer.decode(sample_tokens)}")

# Ejecutar el preprocesamiento
train_df, val_df, test_df, class_weights, label_mapping = load_and_preprocess_data()

# Inicializar el tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
analyze_text_lengths(train_df, tokenizer)

# Crear datasets
train_dataset, val_dataset, test_dataset = create_datasets(
    train_df, val_df, test_df, tokenizer
)

In [3]:
def setup_training_args():
    return TrainingArguments(
        output_dir='/content/drive/MyDrive/IronHack/Proyecto4/nlp-project/models/checkpoints',
        num_train_epochs=4,  # Los autores recomiendan entre 2 y 4 épocas
        per_device_train_batch_size=32,  # Para secuencias cortas, batch de 32 funciona bien
        per_device_eval_batch_size=32,
        warmup_ratio=0.1,
        weight_decay=0.01,  # Según el análisis, weight_decay es el segundo parámetro más importante
        learning_rate=2e-5,  # Valor base recomendado para BERT
        logging_dir='/content/drive/MyDrive/IronHack/Proyecto4/nlp-project/logs',
        logging_steps=100,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,  # Solo guardar los 2 mejores checkpoints
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        gradient_accumulation_steps=2,
        fp16=True,
        max_grad_norm=1.0,
        report_to="none"
    )

# Clase personalizada de Trainer
class CustomTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits.view(-1, 3), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels,
        preds,
        average='weighted'
    )
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
def train_and_evaluate_model():
    logging.info("Iniciando configuración del modelo y entrenamiento...")

    # Configurar el modelo BERT
    num_labels = 3
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=num_labels,
        problem_type="single_label_classification"
    ).to(device)

    # Configurar early stopping
    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=3,
        early_stopping_threshold=0.01
    )

    # Configurar argumentos de entrenamiento
    training_args = setup_training_args()

    # Crear trainer
    trainer = CustomTrainer(
        class_weights=class_weights,
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping_callback]
    )

    # Entrenar el modelo
    logging.info("Iniciando entrenamiento...")
    trainer.train()

    # Evaluar el modelo
    logging.info("Evaluando el modelo...")
    eval_results = trainer.evaluate()

    return trainer, model, eval_results

def hyperparameter_search(base_args):
    logging.info("Iniciando búsqueda de hiperparámetros...")

    # Reducimos épocas para la búsqueda
    base_args.num_train_epochs = 2

    param_combinations = [
        {'learning_rate': 2e-5, 'weight_decay': 0.01},
        {'learning_rate': 3e-5, 'weight_decay': 0.1},
        {'learning_rate': 5e-5, 'weight_decay': 0.01}
    ]

    results = []
    for params in param_combinations:
        logging.info(f"Probando parámetros: {params}")

        current_args = dataclasses.replace(
            base_args,
            learning_rate=params['learning_rate'],
            weight_decay=params['weight_decay']
        )

        temp_trainer = CustomTrainer(
            class_weights=class_weights,
            model=model,
            args=current_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics
        )

        temp_trainer.train()
        eval_results = temp_trainer.evaluate()

        results.append({
            'params': params,
            'results': eval_results
        })

        logging.info(f"Resultados para parámetros {params}: {eval_results}")

    return results

# Ejecutar entrenamiento y evaluación
trainer, model, eval_results = train_and_evaluate_model()
hyperparam_results = hyperparameter_search(trainer.args)

In [ ]:
def visualize_training_metrics(trainer):
    logging.info("Generando visualizaciones de métricas de entrenamiento...")

    # Obtener histórico de entrenamiento
    history = trainer.state.log_history

    # Extraer métricas
    train_loss = [x['loss'] for x in history if 'loss' in x]
    eval_loss = [x['eval_loss'] for x in history if 'eval_loss' in x]

    # Visualizar curvas de aprendizaje
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='Training Loss')
    plt.title('Training Loss over Time')
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(eval_loss, label='Validation Loss')
    plt.title('Validation Loss over Time')
    plt.xlabel('Evaluation Steps')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(trainer, test_dataset):
    logging.info("Generando matriz de confusión...")

    predictions = trainer.predict(test_dataset)
    preds = predictions.predictions.argmax(-1)
    labels = predictions.label_ids

    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=['Positive', 'Neutral', 'Negative'],
        yticklabels=['Positive', 'Neutral', 'Negative']
    )
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    return cm

def analyze_errors(trainer, test_dataset, tokenizer):
    logging.info("Analizando errores de predicción...")

    predictions = trainer.predict(test_dataset)
    preds = predictions.predictions.argmax(-1)
    labels = predictions.label_ids

    incorrect_indices = np.where(preds != labels)[0]
    print("\nAnálisis de errores más comunes:")

    for idx in incorrect_indices[:5]:
        text = test_dataset[idx]['input_ids']
        decoded_text = tokenizer.decode(text, skip_special_tokens=True)
        print(f"\nTexto: {decoded_text[:100]}...")
        print(f"Predicción: {preds[idx]}")
        print(f"Real: {labels[idx]}")

# Ejecutar análisis y visualizaciones
visualize_training_metrics(trainer)
cm = plot_confusion_matrix(trainer, test_dataset)
analyze_errors(trainer, test_dataset, tokenizer)

In [ ]:
def test_model_robustness(trainer, test_samples):
    logging.info("Iniciando pruebas de robustez del modelo...")

    results = []
    for text in test_samples:
        variations = [
            text.lower(),  # Minúsculas
            text.upper(),  # Mayúsculas
            ' '.join(text.split()[:len(text.split())//2]),  # Texto truncado
            text + ' ' * 50,  # Espacios extra
            text.replace('.', '').replace(',', ''),  # Sin puntuación
            ' '.join(text.split()[::-1]),  # Palabras en orden inverso
            text.replace(' ', '  '),  # Doble espacio
            ''.join(char + ' ' for char in text)  # Espacios entre caracteres
        ]

        variation_results = []
        for variant in variations:
            inputs = tokenizer(
                variant,
                return_tensors='pt',
                truncation=True,
                padding=True,
                max_length=128
            ).to(device)

            with torch.no_grad():
                outputs = trainer.model(**inputs)
                pred = outputs.logits.argmax(-1).item()
                confidence = torch.softmax(outputs.logits, dim=1).max().item()
                variation_results.append({
                    'prediction': pred,
                    'confidence': confidence
                })

        # Calcular consistencia y confianza promedio
        predictions = [r['prediction'] for r in variation_results]
        confidences = [r['confidence'] for r in variation_results]
        consistency = len(set(predictions)) == 1
        avg_confidence = sum(confidences) / len(confidences)

        results.append({
            'original': text,
            'predictions': predictions,
            'confidences': confidences,
            'consistent': consistency,
            'avg_confidence': avg_confidence
        })

    return results

def analyze_robustness_results(robustness_results):
    logging.info("Analizando resultados de pruebas de robustez...")

    # Calcular estadísticas generales
    total_samples = len(robustness_results)
    consistent_samples = sum(1 for r in robustness_results if r['consistent'])
    consistency_rate = consistent_samples / total_samples

    avg_confidence = sum(r['avg_confidence'] for r in robustness_results) / total_samples

    print("\nResultados de Pruebas de Robustez:")
    print(f"Total de muestras analizadas: {total_samples}")
    print(f"Muestras con predicciones consistentes: {consistent_samples}")
    print(f"Tasa de consistencia: {consistency_rate:.2%}")
    print(f"Confianza promedio: {avg_confidence:.2%}")

    # Visualizar distribución de confianza
    plt.figure(figsize=(10, 6))
    confidences = [r['avg_confidence'] for r in robustness_results]
    plt.hist(confidences, bins=20)
    plt.title('Distribución de Confianza en Predicciones')
    plt.xlabel('Confianza')
    plt.ylabel('Frecuencia')
    plt.show()

    return {
        'consistency_rate': consistency_rate,
        'avg_confidence': avg_confidence,
        'total_samples': total_samples,
        'consistent_samples': consistent_samples
    }

# Ejecutar pruebas de robustez
test_samples = test_df['reviews.text'].head(10).tolist()  # Usar 10 muestras para prueba
robustness_results = test_model_robustness(trainer, test_samples)
robustness_metrics = analyze_robustness_results(robustness_results)

In [ ]:
def save_model_and_artifacts(trainer, model, tokenizer, config):
    logging.info("Guardando modelo y artefactos...")

    # Crear directorios si no existen
    base_path = '/content/drive/MyDrive/IronHack/Proyecto4/nlp-project'
    model_path = f'{base_path}/models/bert_sentiment'
    os.makedirs(model_path, exist_ok=True)

    # Guardar modelo y tokenizer
    trainer.save_model(model_path)
    tokenizer.save_pretrained(f'{model_path}/tokenizer')

    # Guardar configuración
    config_path = f'{model_path}/config.json'
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=4)

    logging.info(f"Modelo y artefactos guardados en: {model_path}")

    return model_path

def generate_final_report(trainer, eval_results, robustness_metrics, hyperparam_results):
    logging.info("Generando reporte final...")

    report = {
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'model_performance': {
            'accuracy': eval_results['eval_accuracy'],
            'f1': eval_results['eval_f1'],
            'precision': eval_results['eval_precision'],
            'recall': eval_results['eval_recall']
        },
        'robustness_analysis': robustness_metrics,
        'hyperparameter_search': hyperparam_results,
        'training_config': {
            'batch_size': trainer.args.per_device_train_batch_size,
            'learning_rate': trainer.args.learning_rate,
            'epochs': trainer.args.num_train_epochs,
            'weight_decay': trainer.args.weight_decay
        }
    }

    # Guardar reporte
    report_path = 'model_report.json'
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=4)

    # Imprimir resumen
    print("\n=== REPORTE FINAL ===")
    print("\nRendimiento del Modelo:")
    for metric, value in report['model_performance'].items():
        print(f"{metric}: {value:.4f}")

    print("\nAnálisis de Robustez:")
    print(f"Tasa de consistencia: {robustness_metrics['consistency_rate']:.2%}")
    print(f"Confianza promedio: {robustness_metrics['avg_confidence']:.2%}")

    logging.info(f"Reporte final guardado en: {report_path}")
    return report

# Configuración final para guardar
config = {
    'label_mapping': label_mapping,
    'max_length': 128,
    'model_type': 'bert-base-uncased',
    'num_labels': 3,
    'class_weights': class_weights.tolist()
}

# Guardar modelo y generar reporte
model_path = save_model_and_artifacts(trainer, model, tokenizer, config)
final_report = generate_final_report(trainer, eval_results, robustness_metrics, hyperparam_results)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IronHack/Proyecto4/nlp-project/data/processed/consolidated_reviews.csv')

# Análisis de Ratings
print("Distribución de Ratings:")
rating_dist = df['reviews.rating'].value_counts().sort_index()
print(rating_dist)

# Calcular estadísticas básicas de ratings
print("\nEstadísticas de Ratings:")
print(df['reviews.rating'].describe())

# Análisis de Categorías
# Primero necesitamos separar las categorías ya que están en formato 'cat1,cat2,cat3'
all_categories = df['categories'].str.split(',').explode()
print("\nTop 10 Categorías más frecuentes:")
print(all_categories.value_counts().head(10))

# Número total de categorías únicas
print(f"\nNúmero total de categorías únicas: {len(all_categories.unique())}")

# Visualización de la distribución de ratings
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
rating_dist.plot(kind='bar')
plt.title('Distribución de Ratings')
plt.xlabel('Rating')
plt.ylabel('Número de Reviews')
plt.show()

In [ ]:
# Obtener todas las categorías únicas y su frecuencia
all_categories = df['categories'].str.split(',').explode()
all_categories = all_categories.str.strip().str.lower()  # Normalizar

print("Todas las categorías únicas y su frecuencia:")
print(all_categories.value_counts().head(50))

# Ver también algunos patrones comunes
print("\nPatrones comunes en nombres de categorías:")
patterns = {
    'tablet': all_categories[all_categories.str.contains('tablet', case=False)].unique(),
    'electronic': all_categories[all_categories.str.contains('electronic', case=False)].unique(),
    'computer': all_categories[all_categories.str.contains('computer', case=False)].unique(),
    'health': all_categories[all_categories.str.contains('health', case=False)].unique(),
    'home': all_categories[all_categories.str.contains('home', case=False)].unique()
}

for pattern, categories in patterns.items():
    print(f"\nCategorías que contienen '{pattern}':")
    for cat in categories:
        print(f"- {cat}")

In [12]:
def consolidate_categories(category_string):
    categories = set(cat.lower().strip() for cat in category_string.split(','))

    # Tablets & E-readers (mayor presencia)
    if any(cat for cat in categories if 'tablet' in cat or 'kindle' in cat):
        return 'Tablets & E-readers'

    # Amazon Devices (categoría específica importante)
    elif any(cat for cat in categories if 'amazon device' in cat or 'echo' in cat or 'voice assistant' in cat):
        return 'Amazon Devices & Assistants'

    # Smart Home (separado de home general)
    elif any(cat for cat in categories if 'smart home' in cat or 'automation' in cat or 'security' in cat):
        return 'Smart Home & Security'

    # Home Entertainment
    elif any(cat for cat in categories if 'theater' in cat or 'audio' in cat or 'tv' in cat):
        return 'Home Entertainment'

    # Electronics (general)
    elif any(cat for cat in categories if 'electronic' in cat):
        return 'Electronics (General)'

    # Health & Personal Care
    elif any(cat for cat in categories if 'health' in cat or 'personal care' in cat):
        return 'Health & Personal Care'

    # Home & Household
    elif any(cat for cat in categories if 'home' in cat or 'household' in cat):
        return 'Home & Household'

    # Others
    else:
        return 'Others'

In [ ]:
# Aplicar la consolidación de categorías
df['consolidated_category'] = df['categories'].apply(consolidate_categories)

# Ver la distribución de las categorías consolidadas
print("Distribución de Categorías Consolidadas:")
print(df['consolidated_category'].value_counts())

# Ver la distribución de ratings dentro de cada categoría consolidada
print("\nDistribución de Ratings por Categoría Consolidada:")
category_rating_dist = pd.crosstab(df['consolidated_category'], df['reviews.rating'])
print(category_rating_dist)

# Verificar algunos ejemplos de cada categoría
print("\nEjemplos de productos en cada categoría:")
for category in df['consolidated_category'].unique():
    sample = df[df['consolidated_category'] == category][['name', 'reviews.rating', 'reviews.text']].iloc[0]
    print(f"\n{category}:")
    print(f"Producto: {sample['name']}")
    print(f"Rating: {sample['reviews.rating']}")
    print(f"Review: {sample['reviews.text'][:100]}...")  # Primeros 100 caracteres

In [ ]:
def consolidate_categories_final(category):
    if category in ['Home Entertainment', 'Home & Household', 'Others']:
        return 'Electronics (General)'  # Consolidar categorías pequeñas
    return category

# Aplicar la consolidación final
df['final_category'] = df['consolidated_category'].apply(consolidate_categories_final)

# Verificar la distribución final
print("Distribución Final de Categorías:")
print(df['final_category'].value_counts())

In [ ]:
# Primero agrupemos los ratings
def group_ratings(rating):
    if rating <= 2:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    else:
        return 'positive'

# Aplicar el agrupamiento
df['rating_group'] = df['reviews.rating'].apply(group_ratings)

# Crear los grupos para summarization
groups = df.groupby(['final_category', 'rating_group'])

# Ver la distribución final
print("Distribución de reviews por categoría y grupo de rating:")
print(pd.crosstab(df['final_category'], df['rating_group']))

# Preparar datos para summarization
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Inicializar el modelo
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)

# Verificar un grupo de ejemplo
sample_category = 'Tablets & E-readers'
sample_rating = 'positive'
sample_reviews = df[
    (df['final_category'] == sample_category) &
    (df['rating_group'] == sample_rating)
]['reviews.text'].head()

print(f"\nEjemplo de reviews para {sample_category} - {sample_rating}:")
print(sample_reviews)

In [ ]:
def generate_summary(reviews, max_length=150):
    # Preparar el prompt para T5
    prompt = "summarize: " + " ".join(reviews)

    # Tokenizar
    inputs = tokenizer.encode(prompt, return_tensors='pt', max_length=1024,
                            truncation=True).to(device)

    # Generar resumen
    summary_ids = model.generate(inputs,
                               max_length=max_length,
                               num_beams=4,
                               length_penalty=2.0,
                               early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Crear diccionario para almacenar los resúmenes
summaries = {}

# Generar resúmenes para cada combinación de categoría y rating
for category in df['final_category'].unique():
    summaries[category] = {}
    for rating in ['negative', 'neutral', 'positive']:
        reviews = df[
            (df['final_category'] == category) &
            (df['rating_group'] == rating)
        ]['reviews.text'].head(50)  # Tomamos 50 reviews para el resumen

        if len(reviews) > 0:
            summary = generate_summary(reviews)
            summaries[category][rating] = summary

            print(f"\nResumen para {category} - {rating}:")
            print(summary)

In [ ]:
# Pruebas de robustez del modelo
def test_model_robustness(trainer, samples):
    results = []
    for text in samples:
        variations = [
            text.lower(),  # Minúsculas
            text.upper(),  # Mayúsculas
            ' '.join(text.split()[:len(text.split())//2]),  # Texto truncado
            text + ' ' * 50,  # Espacios extra
            text.replace('.', '').replace(',', ''),  # Sin puntuación
            ' '.join(text.split()[::-1])  # Palabras en orden inverso
        ]

        variation_results = []
        for variant in variations:
            inputs = tokenizer(
                variant,
                return_tensors='pt',
                truncation=True,
                padding=True,
                max_length=128
            ).to(device)

            with torch.no_grad():
                outputs = trainer.model(**inputs)
                pred = outputs.logits.argmax(-1).item()
                variation_results.append(pred)

        # Calcular consistencia
        consistency = len(set(variation_results)) == 1
        results.append({
            'original': text,
            'predictions': variation_results,
            'consistent': consistency
        })

    return results

# Probar con algunas muestras
test_samples = test_df['reviews.text'].head(5).tolist()
robustness_results = test_model_robustness(trainer, test_samples)

# Mostrar resultados
print("\nResultados de pruebas de robustez:")
for i, result in enumerate(robustness_results):
    print(f"\nMuestra {i+1}:")
    print(f"Texto original: {result['original'][:100]}...")
    print(f"Consistente: {'Sí' if result['consistent'] else 'No'}")
    print(f"Predicciones para variaciones: {result['predictions']}")

In [ ]:
# Guardar modelos y configuraciones para el dashboard
import json
import pickle
import os
print("Guardando modelos y configuraciones...")

# Directorio base para modelos y configuraciones
base_path = '/content/drive/MyDrive/IronHack/Proyecto4/nlp-project'

# Crear directorios si no existen
os.makedirs(f'{base_path}/models/bert_sentiment', exist_ok=True)
os.makedirs(f'{base_path}/models/bert_tokenizer', exist_ok=True)

# 1. Guardar el modelo BERT y tokenizer
print("Guardando modelo BERT y tokenizer...")
model_path = f'{base_path}/models/bert_sentiment'
tokenizer_path = f'{base_path}/models/bert_tokenizer'
model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

# 2. Guardar configuraciones y mapeos
print("Guardando configuraciones...")
config = {
    'label_mapping': {str(k): v for k, v in label_mapping.items()},  # Convertir a formato serializable
    'max_length': 128,
    'model_path': model_path,
    'tokenizer_path': tokenizer_path
}

with open(f'{base_path}/models/config.json', 'w') as f:
    json.dump(config, f)

# 3. Guardar funciones de consolidación de categorías
print("Guardando funciones de procesamiento...")
with open(f'{base_path}/models/category_processing.pkl', 'wb') as f:
    pickle.dump({
        'consolidate_categories': consolidate_categories,
        'consolidate_categories_final': consolidate_categories_final
    }, f)

# 4. Guardar los summaries generados
print("Guardando summaries...")
with open(f'{base_path}/models/summaries.pkl', 'wb') as f:
    pickle.dump(summaries, f)

print("¡Guardado completado!")

In [ ]:
# Generar reporte final
def generate_final_report(trainer, test_results, robustness_results, hyperparam_results):
    report = {
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'model_performance': {
            'accuracy': test_results['eval_accuracy'],
            'f1': test_results['eval_f1'],
            'precision': test_results['eval_precision'],
            'recall': test_results['eval_recall']
        },
        'hyperparameter_search': hyperparam_results,
        'robustness_analysis': {
            'samples_tested': len(robustness_results),
            'consistency_rate': sum(r['consistent'] for r in robustness_results) / len(robustness_results)
        }
    }

    # Guardar reporte
    with open('model_report.json', 'w') as f:
        json.dump(report, f, indent=4)

    # Imprimir resumen
    print("=== REPORTE FINAL ===")
    print(f"\nRendimiento del modelo:")
    for metric, value in report['model_performance'].items():
        print(f"{metric}: {value:.4f}")

    print(f"\nTasa de consistencia en pruebas de robustez: {report['robustness_analysis']['consistency_rate']:.2%}")

# Generar reporte final
generate_final_report(trainer, eval_results, robustness_results, hyperparam_results)

In [ ]:
def generate_summaries_by_rating(df, model, tokenizer, device, max_reviews_per_rating=50):
    """
    Genera resúmenes por categoría y rating (1-5)
    """
    summaries = {}

    for category in df['final_category'].unique():
        summaries[category] = {}

        for rating in range(1, 6):  # Ratings del 1 al 5
            reviews = df[
                (df['final_category'] == category) &
                (df['reviews.rating'] == rating)
            ]['reviews.text'].head(max_reviews_per_rating)

            if len(reviews) > 0:
                # Preparar el prompt para T5
                prompt = f"summarize reviews for {category} with {rating} stars: " + " ".join(reviews)

                # Tokenizar
                inputs = tokenizer.encode(
                    prompt,
                    return_tensors='pt',
                    max_length=1024,
                    truncation=True
                ).to(device)

                # Generar resumen
                summary_ids = model.generate(
                    inputs,
                    max_length=150,
                    num_beams=4,
                    length_penalty=2.0,
                    early_stopping=True
                )

                summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
                summaries[category][rating] = {
                    'summary': summary,
                    'review_count': len(reviews),
                    'avg_length': reviews.str.len().mean()
                }

                print(f"\nResumen para {category} - {rating} estrellas:")
                print(f"Basado en {len(reviews)} reviews")
                print(summary)

    return summaries

# Análisis adicional de la distribución de ratings
def analyze_rating_distribution(df):
    """
    Analiza la distribución de ratings por categoría
    """
    # Distribución general
    print("Distribución general de ratings:")
    print(df['reviews.rating'].value_counts().sort_index())

    # Distribución por categoría
    rating_dist = pd.crosstab(
        df['final_category'],
        df['reviews.rating']
    ).apply(lambda x: x/x.sum() * 100, axis=1)

    # Visualización
    plt.figure(figsize=(12, 6))
    rating_dist.plot(kind='bar', stacked=True)
    plt.title('Distribución de Ratings por Categoría (%)')
    plt.xlabel('Categoría')
    plt.ylabel('Porcentaje')
    plt.legend(title='Rating')
    plt.tight_layout()
    plt.show()

    return rating_dist

# Estructura de datos para el dashboard
def prepare_dashboard_data(summaries, rating_dist):
    """
    Prepara los datos en un formato útil para el dashboard
    """
    dashboard_data = {
        'summaries': summaries,
        'rating_distribution': rating_dist.to_dict(),
        'metadata': {
            'categories': list(summaries.keys()),
            'ratings': list(range(1, 6)),
            'last_updated': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
    }

    return dashboard_data

# Ejemplo de uso
summaries = generate_summaries_by_rating(df, model, tokenizer, device)
rating_dist = analyze_rating_distribution(df)
dashboard_data = prepare_dashboard_data(summaries, rating_dist)

# Guardar para el dashboard
with open(f'{base_path}/models/dashboard_data.json', 'w') as f:
    json.dump(dashboard_data, f, indent=4)

In [ ]:
!pip install rouge_score nltk

from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
import evaluate
import ssl

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

In [ ]:




def evaluate_summaries(original_reviews, generated_summary):
    """
    Evalúa la calidad del resumen usando ROUGE y BLEU
    """
    # Inicializar ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Preparar texto de referencia (concatenamos algunos reviews como referencia)
    reference_text = ' '.join(original_reviews[:5])  # Usamos los primeros 5 reviews como referencia

    # Calcular scores ROUGE
    rouge_scores = scorer.score(reference_text, generated_summary)

    # Preparar para BLEU
    # Tokenizamos el resumen generado y la referencia
    reference_tokens = [nltk.word_tokenize(review.lower()) for review in original_reviews[:5]]
    candidate_tokens = nltk.word_tokenize(generated_summary.lower())

    # Calcular BLEU score
    smoothing = SmoothingFunction().method1
    bleu_score = sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smoothing)

    return {
        'rouge1': rouge_scores['rouge1'].fmeasure,
        'rouge2': rouge_scores['rouge2'].fmeasure,
        'rougeL': rouge_scores['rougeL'].fmeasure,
        'bleu': bleu_score
    }

def generate_summaries_with_evaluation(df, model, tokenizer, device, max_reviews_per_rating=50):
    """
    Versión mejorada que incluye evaluación de calidad de los resúmenes
    """
    summaries = {}

    for category in df['final_category'].unique():
        summaries[category] = {}

        for rating in range(1, 6):
            reviews = df[
                (df['final_category'] == category) &
                (df['reviews.rating'] == rating)
            ]['reviews.text']

            if len(reviews) > 0:
                # Tomar muestra para el resumen
                sample_reviews = reviews.head(max_reviews_per_rating)

                # Generar resumen
                prompt = f"summarize reviews for {category} with {rating} stars: " + " ".join(sample_reviews)
                inputs = tokenizer.encode(prompt, return_tensors='pt', max_length=1024, truncation=True).to(device)

                summary_ids = model.generate(
                    inputs,
                    max_length=150,
                    num_beams=4,
                    length_penalty=2.0,
                    early_stopping=True
                )

                summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

                # Evaluar calidad del resumen
                quality_metrics = evaluate_summaries(sample_reviews.tolist(), summary)

                summaries[category][rating] = {
                    'summary': summary,
                    'review_count': len(sample_reviews),
                    'avg_length': sample_reviews.str.len().mean(),
                    'quality_metrics': quality_metrics
                }

                print(f"\nResumen para {category} - {rating} estrellas:")
                print(f"Basado en {len(sample_reviews)} reviews")
                print(summary)
                print("\nMétricas de calidad:")
                print(f"ROUGE-1: {quality_metrics['rouge1']:.3f}")
                print(f"ROUGE-2: {quality_metrics['rouge2']:.3f}")
                print(f"ROUGE-L: {quality_metrics['rougeL']:.3f}")
                print(f"BLEU: {quality_metrics['bleu']:.3f}")

    return summaries

# Función para analizar las métricas de calidad
def analyze_summary_quality(summaries):
    """
    Analiza las métricas de calidad de todos los resúmenes
    """
    quality_data = {
        'category': [],
        'rating': [],
        'rouge1': [],
        'rouge2': [],
        'rougeL': [],
        'bleu': []
    }

    for category in summaries:
        for rating in summaries[category]:
            metrics = summaries[category][rating]['quality_metrics']
            quality_data['category'].append(category)
            quality_data['rating'].append(rating)
            quality_data['rouge1'].append(metrics['rouge1'])
            quality_data['rouge2'].append(metrics['rouge2'])
            quality_data['rougeL'].append(metrics['rougeL'])
            quality_data['bleu'].append(metrics['bleu'])

    quality_df = pd.DataFrame(quality_data)

    # Visualización de métricas
    plt.figure(figsize=(15, 6))

    plt.subplot(1, 2, 1)
    sns.boxplot(data=quality_df.melt(id_vars=['category'],
                                   value_vars=['rouge1', 'rouge2', 'rougeL']),
                x='variable', y='value')
    plt.title('Distribución de métricas ROUGE')

    plt.subplot(1, 2, 2)
    sns.boxplot(data=quality_df, x='rating', y='bleu')
    plt.title('Distribución de BLEU score por rating')

    plt.tight_layout()
    plt.show()

    return quality_df

# Ejemplo de uso
summaries = generate_summaries_with_evaluation(df, model, tokenizer, device)
quality_analysis = analyze_summary_quality(summaries)